<div style="text-align: center;"><br>
<img src="https://assets-global.website-files.com/62b9d45fb3f64842a96c9686/62d84db4aeb2f6552f3a2f78_Quantinuum%20Logo__horizontal%20blue.svg" width="200" height="200" /></div>

# Mid-Circuit Measurement

This notebook contains an example using mid-circuit measurement using the Quantinuum machines.

## Repetition Code Circuit

The use of mid-circuit measurement is straightforward, note the use of `measure` and `reset` on the ancilla qubits. This example also utlizes conditional logic available with Quantinuum devices as well as Registers and IDs available in `pytket`. See [Classical and conditional operations](https://tket.quantinuum.com/user-manual/manual_circuit.html#classical-and-conditional-operations) and [Registers and IDs](https://cqcl.github.io/pytket/manual/manual_circuit.html#registers-and-ids) for additional examples.

In [1]:
from pytket.circuit import Circuit, Qubit, Bit, OpType, reg_eq
from pytket.circuit.display import render_circuit_jupyter

Set up Repetition Code with mid-circuit measurement and corrections

2    1    0 = data: data qubits<br>
*----*----*<br>
  ZZ   ZZ<br>
  1    0    = syndromes<br>
  0    0    = ancillas

Set up circuit object

In [2]:
circuit = Circuit(name="Repetition Code")

Reserve registries

Add qubit register, the data qubits

In [3]:
data = circuit.add_q_register("data", 3)

Add qubit register, the ancilla qubit

In [4]:
ancilla = circuit.add_q_register("anc", 1)

Add classical registers for the syndromes

In [5]:
syndrome = circuit.add_c_register("syndrome", 2)

Add classical registers for the output

In [6]:
output = circuit.add_c_register("output", 3)

Prepare the logical state<br>
Qubits always start in |0> and logical |0> == |000>.<br>
So we already start in logical |0>.

Syndrome Extraction

In [7]:
circuit.add_gate(OpType.Reset, ancilla)
circuit.CX(data[0], ancilla[0])
circuit.CX(data[1], ancilla[0])
circuit.Measure(ancilla[0], syndrome[0])

[Reset anc[0]; CX data[0], anc[0]; CX data[1], anc[0]; Measure anc[0] --> syndrome[0]; ]

In [8]:
render_circuit_jupyter(circuit)

Syndrome Extraction

In [9]:
circuit.add_gate(OpType.Reset, ancilla)
circuit.CX(data[1], ancilla[0])
circuit.CX(data[2], ancilla[0])
circuit.Measure(ancilla[0], syndrome[1])

[Reset anc[0]; CX data[0], anc[0]; CX data[1], anc[0]; Measure anc[0] --> syndrome[0]; Reset anc[0]; CX data[1], anc[0]; CX data[2], anc[0]; Measure anc[0] --> syndrome[1]; ]

In [10]:
render_circuit_jupyter(circuit)

Correction<br>
if(syndromes==1) -> 01 -> check 0 bad -> X on qubit 0

In [11]:
circuit.X(data[0], condition=reg_eq(syndrome, 1))

[Reset anc[0]; CX data[0], anc[0]; CX data[1], anc[0]; Measure anc[0] --> syndrome[0]; Reset anc[0]; CX data[1], anc[0]; CX data[2], anc[0]; Measure anc[0] --> syndrome[1]; RangePredicate([1,1]) syndrome[0], syndrome[1], tk_SCRATCH_BIT[0]; IF ([tk_SCRATCH_BIT[0]] == 1) THEN X data[0]; ]

In [12]:
render_circuit_jupyter(circuit)

if(syndromes==2) -> 10 -> check 1 bad -> X on qubit 2

In [13]:
circuit.X(data[2], condition=reg_eq(syndrome, 2))

[Reset anc[0]; CX data[0], anc[0]; CX data[1], anc[0]; Measure anc[0] --> syndrome[0]; Reset anc[0]; CX data[1], anc[0]; CX data[2], anc[0]; Measure anc[0] --> syndrome[1]; RangePredicate([1,1]) syndrome[0], syndrome[1], tk_SCRATCH_BIT[0]; RangePredicate([2,2]) syndrome[0], syndrome[1], tk_SCRATCH_BIT[1]; IF ([tk_SCRATCH_BIT[0]] == 1) THEN X data[0]; IF ([tk_SCRATCH_BIT[1]] == 1) THEN X data[2]; ]

In [14]:
render_circuit_jupyter(circuit)

if(syndromes==3) -> 11 -> check 1 and 2 bad -> X on qubit 1

In [15]:
circuit.X(data[1], condition=reg_eq(syndrome, 3))

[Reset anc[0]; CX data[0], anc[0]; CX data[1], anc[0]; Measure anc[0] --> syndrome[0]; Reset anc[0]; CX data[1], anc[0]; CX data[2], anc[0]; Measure anc[0] --> syndrome[1]; RangePredicate([1,1]) syndrome[0], syndrome[1], tk_SCRATCH_BIT[0]; RangePredicate([2,2]) syndrome[0], syndrome[1], tk_SCRATCH_BIT[1]; RangePredicate([3,3]) syndrome[0], syndrome[1], tk_SCRATCH_BIT[2]; IF ([tk_SCRATCH_BIT[0]] == 1) THEN X data[0]; IF ([tk_SCRATCH_BIT[2]] == 1) THEN X data[1]; IF ([tk_SCRATCH_BIT[1]] == 1) THEN X data[2]; ]

In [16]:
render_circuit_jupyter(circuit)

Measure out data qubits

In [17]:
circuit.Measure(data[0], output[0])
circuit.Measure(data[1], output[1])
circuit.Measure(data[2], output[2])

[Reset anc[0]; CX data[0], anc[0]; CX data[1], anc[0]; Measure anc[0] --> syndrome[0]; Reset anc[0]; CX data[1], anc[0]; CX data[2], anc[0]; Measure anc[0] --> syndrome[1]; RangePredicate([1,1]) syndrome[0], syndrome[1], tk_SCRATCH_BIT[0]; RangePredicate([2,2]) syndrome[0], syndrome[1], tk_SCRATCH_BIT[1]; RangePredicate([3,3]) syndrome[0], syndrome[1], tk_SCRATCH_BIT[2]; IF ([tk_SCRATCH_BIT[0]] == 1) THEN X data[0]; IF ([tk_SCRATCH_BIT[2]] == 1) THEN X data[1]; IF ([tk_SCRATCH_BIT[1]] == 1) THEN X data[2]; Measure data[0] --> output[0]; Measure data[1] --> output[1]; Measure data[2] --> output[2]; ]

In [18]:
render_circuit_jupyter(circuit)

## Select Device

Login to the Quantinuum API using your credentials and check the device status.

In [19]:
from pytket.extensions.quantinuum import QuantinuumBackend

In [20]:
machine = "H1-1E"
backend = QuantinuumBackend(device_name=machine)
backend.login()

In [21]:
print(machine, "status:", backend.device_state(device_name=machine))

H1-1E status: online


### Circuit Compilation

`pytket` includes many features for optimizing circuits. This includes reducing the number of gates where possible and resynthesizing circuits for a quantum computer's native gate set. See the `pytket` [User Manual](https://cqcl.github.io/pytket/manual/index.html) for more information on all the options that are available.

Here the circuit is compiled with `get_compiled_circuit`, which includes optimizing the gates and resynthesizing the circuit to Quantinuum's native gate set. The `optimisation_level` sets the level of optimisation to perform during compilation, check [Default Compilation](https://cqcl.github.io/pytket-quantinuum/api/index.html#default-compilation) in the pytket-quantinuum documentation for more details.

In [22]:
compiled_circuit = backend.get_compiled_circuit(circuit, optimisation_level=1)

In [23]:
render_circuit_jupyter(compiled_circuit)

## Submit and Run the Circuit

In [25]:
n_shots = 100
print(
    "Cost in HQCs:",
    backend.cost(compiled_circuit, n_shots=n_shots, syntax_checker="H1-1SC"),
)
handle = backend.process_circuit(compiled_circuit, n_shots=n_shots)
print(handle)
status = backend.circuit_status(handle)
print(status)

Cost in HQCs: 7.04
('477e3c727a2d4670b79bbcecdbd29d9c', 'null', 5, '[["output", 0], ["output", 1], ["output", 2], ["syndrome", 0], ["syndrome", 1]]')
CircuitStatus(status=<StatusEnum.QUEUED: 'Circuit is queued.'>, message='{"name": "Repetition Code", "submit-date": "2024-03-27T05:39:21.310198", "result-date": null, "queue-position": null, "cost": "7.04", "error": null}', error_detail=None, completed_time=None, queued_time=None, submitted_time=None, running_time=None, cancelled_time=None, error_time=None, queue_position=None)


In [26]:
import json

In [28]:
result = backend.get_result(handle)

In [29]:
with open("pytket_mcmr_example.json", "w") as file:
    json.dump(result.to_dict(), file)

## Analyze Results

We will now take the raw results and apply a majority vote to determine how many times we got 0 vs 1.

First, define a majority vote function.

In [30]:
def majority(result):
    """Returns whether the output should be considered a 0 or 1."""
    if result.count(0) > result.count(1):
        return 0
    elif result.count(0) < result.count(1):
        return 1
    else:
        raise Exception("count(0) should not equal count(1)")

Now process the output:

In [31]:
result_output_cnts = result.get_counts([output[i] for i in range(output.size)])

In [32]:
result_output_cnts

Counter({(0, 0, 0): 99, (1, 0, 0): 1})

Here, determine how many times 0 vs 1 was observed using the majority vote function.

In [33]:
zeros = 0  # Counts the shots with majority zeros
ones = 0  # Counts the shots with majority ones

In [34]:
for out in result_output_cnts:
    m = majority(out)
    if m == 0:
        zeros += result_output_cnts[out]
    else:
        ones += result_output_cnts[out]

A logical zero was initialized, so our error rate should be number of ones / total number of shots: `ones/shots`

In [35]:
p = ones / n_shots
print(f"The error-rate is: p = {p}")

The error-rate is: p = 0.0


<div align="center"> &copy; 2024 by Quantinuum. All Rights Reserved. </div>